# ADS Project 4: Machine Learning Fairness
## Spring 2023

In [3]:
# ! pip install dalex
import pandas as pd
import numpy as np
import dalex as dx
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss


+ The cleaned COMPAS dataset is provided in `../output/compas-scores-two-years_cleaned.csv`. 

### Section 1: Load Data

In [4]:
df =  pd.read_csv('../output/compas-scores-two-years_cleaned.csv')
df.head(3)

,sex,age_cat,race,priors_count,c_charge_degree,two_year_recid,length_of_stay
0,Male,25 - 45,African-American,-0.733607,F,1,-0.167773
1,Male,< 25,African-American,0.055928,F,1,-0.340654
2,Male,25 - 45,Caucasian,2.029767,F,1,-0.244609


In [5]:
# change categorical data into numerical
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])
df['age_cat'] = le.fit_transform(df['age_cat'])
df['race'] = le.fit_transform(df['race'])
df['c_charge_degree'] = le.fit_transform(df['c_charge_degree'])

Encode categorical variables with numerical variables:
+ `sex`: 1 for male and 0 for female
+ `age_cat`: 2 for > 45, 0 for 25 - 45 and 1 for < 25
+ `race`: 1 for caucasian and 0 for african-american
+ `c_charge_degree`: 0 for F and 1 for M

In [6]:
# 
df['sex'].value_counts(), df['age_cat'].value_counts(), df['race'].value_counts(), df['c_charge_degree'].value_counts()

(1    4751
 0    1164
 Name: sex, dtype: int64,
 0    3378
 1    1281
 2    1256
 Name: age_cat, dtype: int64,
 0    3537
 1    2378
 Name: race, dtype: int64,
 0    3904
 1    2011
 Name: c_charge_degree, dtype: int64)

In [7]:
features = df[['sex', 'age_cat', 'c_charge_degree', 'length_of_stay']]
sensitive = df['race']
target = df['two_year_recid']
X_train, X_test, y_train, y_test, race_train, race_test = \
    train_test_split(features, target, sensitive, test_size=0.3, random_state=6, shuffle = True)

+ **Protected**: Caucasians (i.e., `race == 1`)
+ **Not protected**: African-Americans (i.e., `race == 0`)

In [10]:
df.head()

,sex,age_cat,race,priors_count,c_charge_degree,two_year_recid,length_of_stay
0,1,0,0,-0.733607,0,1,-0.167773
1,1,1,0,0.055928,0,1,-0.340654
2,1,0,1,2.029767,0,1,-0.244609
3,0,0,1,-0.733607,1,0,-0.321445
4,1,1,1,-0.536224,0,1,-0.359864


The p-rule  function is commonly used in evaluate fairness in machine learning model, by checking whether the model's positive predictions are distributed similarly across different sensitive groups. The higher the p-rule, the better the fairness.

In [11]:
# Function to compute p-rule
def p_rule(sensitive_var, y_pred):
    protected = np.where(sensitive_var == 1)[0]
    not_protected = np.where(sensitive_var == 0)[0]
    protected_pred = np.where(y_pred[protected] == 1)
    not_protected_pred = np.where(y_pred[not_protected] == 1)
    protected_percent = protected_pred[0].shape[0]/protected.shape[0]
    not_protected_percent = not_protected_pred[0].shape[0]/not_protected.shape[0]
    ratio = min(protected_percent/not_protected_percent, not_protected_percent/protected_percent)
    
    return ratio, protected_percent, not_protected_percent

### Section 2: Logistic Regression
#### 2.1 Baseline Model
First, we train a baseline model without constaint, and then evaluate it's accuaracy and fairness.

In [12]:
clf = LogisticRegression(random_state= 6).fit(X_train, y_train)
coeff = clf.coef_
intercept = clf.intercept_
optimal_loss = log_loss(y_train, clf.predict_proba(X_train))
print(f'The optimal loss of logistic model is: {optimal_loss}')

The optimal loss of logistic model is: 0.6708237277484401


+ Accuaracy & Fairness check:

In [13]:
results_lr = {"Classifier": ["LR", "LR"], 
              "Set": ["Train", "Test"],
              "Accuracy (%)": [clf.score(X_train, y_train)*100, clf.score(X_test, y_test)*100],
              "P-rule (%)": [p_rule(race_train, clf.predict(X_train))[0]*100, p_rule(race_test, clf.predict(X_test))[0]*100],
              "Protected (%)": [p_rule(race_train, clf.predict(X_train))[1]*100, p_rule(race_test, clf.predict(X_test))[1]*100],
              "Not protected (%)": [p_rule(race_train, clf.predict(X_train))[2]*100, p_rule(race_test, clf.predict(X_test))[2]*100]}
pd.DataFrame(results_lr)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%)
0,LR,Train,56.714976,73.877917,30.640335,41.474281
1,LR,Test,55.098592,77.978155,31.541726,40.449438


#### 2.2 Optimizing classifier accuracy subject to fairness constraints

In [98]:
# Fairness Constraints: Mechanisms for Fair Classification

### Section 3: Support Vector Machine (SVM) 
#### 3.1 Baseline Model

In [14]:
from sklearn import svm
from sklearn.svm import SVC

# Train model
svm_model = SVC(kernel = 'linear', probability = True)
clf= svm_model.fit(X_train, y_train)
optimal_loss = log_loss(y_train, clf.predict_proba(X_train))
print(f'The optimal loss of SVM model is: {optimal_loss}')

The optimal loss of SVM model is: 0.6747212077046003


In [15]:
# Display results
results_svm = {"Classifier": ["SVM", "SVM"],
               "Set": ["Train", "Test"],
               "Accuracy (%)": [clf.score(X_train, y_train)*100, clf.score(X_test, y_test)*100],
               "P-rule (%)": [p_rule(race_train, clf.predict(X_train))[0]*100, p_rule(race_test, clf.predict(X_test))[0]*100],
               "Protected (%)": [p_rule(race_train, clf.predict(X_train))[1]*100, p_rule(race_test, clf.predict(X_test))[1]*100],
               "Not protected (%)": [p_rule(race_train, clf.predict(X_train))[2]*100, p_rule(race_test, clf.predict(X_test))[2]*100]}
pd.DataFrame(results_svm)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%)
0,SVM,Train,58.792271,72.023947,38.360263,53.260429
1,SVM,Test,57.070423,73.475161,40.452617,55.056180


#### 3.2 Optimizing SVM classifier accuracy subject to fairness constraints

In [ ]:
# Fairness Constraints: Mechanisms for Fair Classification

### Section 4: Information Theoretic Measures for Fairness-aware Feature Selection(FFS) 

#### 4.1 Logistic Regression using FFS

#### 4.2 SVM using FFS

### Section 5: Evaluation